In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("crop_production.csv")

In [ ]:
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


In [ ]:
df.isnull().sum()

,0
State_Name,0
District_Name,0
Crop_Year,0
Season,0
Crop,0
Area,0
Production,3730


In [ ]:
df.shape

(246091, 7)

In [ ]:
# remove the null col
df = df.dropna()

In [ ]:
df.isnull().sum()

,0
State_Name,0
District_Name,0
Crop_Year,0
Season,0
Crop,0
Area,0
Production,0


In [ ]:
df.shape

(242361, 7)

In [ ]:
df['Yield'] = df['Production'] / df['Area']

In [ ]:
df['Yield'].describe()

,Yield
count,242361.000000
mean,41.649059
std,817.572839
min,0.000000
25%,0.513514
50%,1.000000
75%,2.355450
max,88000.000000


In [ ]:
#removing the outliers
Q1 = df['Yield'].quantile(0.25)
Q3 = df['Yield'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df[(df['Yield'] >= lower_bound) & (df['Yield'] <= upper_bound)]

In [ ]:
df.shape

(205738, 8)

In [ ]:
df['Yield'].describe()

,Yield
count,205738.000000
mean,1.142188
std,0.985188
min,0.000000
25%,0.469194
50%,0.837607
75%,1.500000
max,5.118052


In [ ]:
len(df['District_Name'].value_counts())

645

In [ ]:
df['Area'] = np.log1p(df['Area'])

In [ ]:
#drop less yiels corps
df = df[df['Yield'] > 2.5]

In [ ]:
df['Season'].value_counts()

,count
Season,
Rabi,6362
Whole Year,5884
Kharif,5647
Summer,2531
Winter,368
Autumn,297


In [ ]:
df.shape

(21089, 8)

In [ ]:
df['Yield'].describe()

,Yield
count,21089.000000
mean,3.447985
std,0.740082
min,2.500055
25%,2.841384
50%,3.227941
75%,4.000000
max,5.118052


In [ ]:
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,Yield
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,4.634729,321.0,3.147059
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,5.176150,641.0,3.642045
6,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Dry ginger,3.610918,100.0,2.777778
8,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Sweet potato,1.791759,15.0,3.000000
9,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Tapioca,3.713572,169.0,4.225000


In [ ]:
df = df.drop(['Production' , 'Crop_Year' , 'Yield'], axis=1)

In [ ]:
df.head()

,State_Name,District_Name,Season,Crop,Area
2,Andaman and Nicobar Islands,NICOBARS,Kharif,Rice,4.634729
3,Andaman and Nicobar Islands,NICOBARS,Whole Year,Banana,5.176150
6,Andaman and Nicobar Islands,NICOBARS,Whole Year,Dry ginger,3.610918
8,Andaman and Nicobar Islands,NICOBARS,Whole Year,Sweet potato,1.791759
9,Andaman and Nicobar Islands,NICOBARS,Whole Year,Tapioca,3.713572


In [ ]:
%pip install catboost
%pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.5 MB/s eta 0:00:00


In [ ]:
# Train the model useing catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
X = df[['State_Name' , 'District_Name' , 'Season','Area']]
y = df['Crop']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
catboost_model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, verbose=0,
                                    cat_features=['State_Name', 'District_Name', 'Season'] , task_type="GPU", devices='0:1') # Specify categorical features
catboost_model.fit(X_train, y_train)

In [ ]:
y_pred_catboost = catboost_model.predict(X_test)
catboost_accuracy = accuracy_score(y_test, y_pred_catboost)
print(f"CatBoost Accuracy: {catboost_accuracy}")


CatBoost Accuracy: 0.6955903271692745


In [ ]:
# funtion to predict top 3 crops
def predict_top_crops(state, district, season, area):
    input_data = pd.DataFrame([[state, district, season, area]], columns=['State_Name', 'District_Name', 'Season', 'Area'])
    input_data['Area'] = np.log1p(input_data['Area'])
    predicted_crops = catboost_model.predict(input_data)
    return predicted_crops



In [ ]:
def predict_top_3_crops(state, district, season, area):
    input_data = pd.DataFrame([[state, district, season, area]],
                              columns=['State_Name', 'District_Name', 'Season', 'Area'])
    input_data['Area'] = np.log1p(input_data['Area'])

    # Get probabilities for each crop
    predicted_probs = catboost_model.predict_proba(input_data)[0]

    # Get indices of top 3 probabilities
    top_3_indices = predicted_probs.argsort()[-3:][::-1]

    # Get the corresponding crop names
    top_3_crops = catboost_model.classes_[top_3_indices]

    return top_3_crops

In [ ]:
# call the prediction funtion
predict_top_3_crops('Madhya Pradesh', 'Gwalior', 'Kharif', 1000)

array(['Maize', 'Rice', 'Barley'], dtype=object)

In [ ]:
# if torch.cuda.is_available():
#     print("GPU is available")
#     catboost_model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, verbose=0, task_type="GPU", devices='0:1')

In [ ]:
# checking gpu available or not
import torch
if torch.cuda.is_available():
    print("GPU is available")

GPU is available


In [ ]:
df['Area'].describe()

,Area
count,21089.000000
mean,7.257939
std,3.234658
min,0.693147
25%,4.727388
50%,7.177019
75%,10.138599
max,13.233549


In [ ]:
catboost_model.save_model("model_useing_Location_Seasion_Area.h5")


In [ ]:
import pickle

# Assuming 'catboost_model' is your trained CatBoostClassifier model
with open('catboost_model.pkl', 'wb') as f:
    pickle.dump(catboost_model, f)


In [ ]:
# prompt: for future caling of the model we need the unique stae , disrict and season name , we need to stre them

import pandas as pd

# Load the dataset (assuming it's already loaded as 'df' in your original code)
# ... (your existing code to load the dataframe)

# Extract unique values
unique_states = df['State_Name'].unique()
unique_districts = df['District_Name'].unique()
unique_seasons = df['Season'].unique()

# Create a dictionary to store the unique values
unique_values = {
    'states': unique_states.tolist(),
    'districts': unique_districts.tolist(),
    'seasons': unique_seasons.tolist()
}

# Save the dictionary to a file (e.g., as a JSON file)
import json
with open('unique_values.json', 'w') as f:
    json.dump(unique_values, f)


In [ ]:
unique_corps = df['Crop'].unique()
print(unique_corps)

['Rice' 'Banana' 'Dry ginger' 'Sweet potato' 'Tapioca' 'Dry chillies'
 'Sugarcane' 'Maize' 'Arecanut' 'Jowar' 'Ragi' 'Turmeric'
 'Beans & Mutter(Vegetable)' 'Citrus Fruit' 'Tomato' 'Bajra' 'Tobacco'
 'Groundnut' 'Mango' 'Gram' 'Cotton(lint)' 'Other Fresh Fruits' 'Lemon'
 'Onion' 'Soyabean' 'Cabbage' 'Potato' 'Rapeseed &Mustard' 'Pome Fruit'
 'Bhindi' 'Ginger' 'Brinjal' 'Oilseeds total' 'Wheat' 'Pulses total'
 'Mesta' 'Coconut ' 'Paddy' 'Jute' 'Peas & beans (Pulses)' 'Black pepper'
 'Sannhamp' 'Arhar/Tur' 'Masoor' 'Barley' 'Other Kharif pulses'
 'Sunflower' 'Garlic' 'Coriander' 'Khesari' 'Horse-gram'
 'Other  Rabi pulses' 'Castor seed' 'Niger seed' 'Other Cereals & Millets'
 'Other Vegetables' 'Urad' 'Small millets' 'Papaya' 'Atcanut (Raw)'
 'Pineapple' 'Tea' 'Jack Fruit' 'Orange' 'Bitter Gourd' 'Cowpea(Lobia)'
 'Cashewnut' 'Guar seed' 'Linseed' 'Grapes' 'Total foodgrain' 'Safflower'
 'Cardamom']
